# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121436e+02     1.593117e+00
 * time: 0.7098240852355957
     1     1.031245e+01     8.866464e-01
 * time: 2.850947141647339
     2    -1.223895e+01     1.033495e+00
 * time: 3.496696949005127
     3    -3.484892e+01     7.410829e-01
 * time: 4.360658168792725
     4    -4.838587e+01     5.926876e-01
 * time: 5.280102014541626
     5    -5.734743e+01     1.999443e-01
 * time: 6.189496994018555
     6    -6.000489e+01     1.176229e-01
 * time: 6.787836074829102
     7    -6.100556e+01     4.914220e-02
 * time: 7.372948169708252
     8    -6.138519e+01     7.200936e-02
 * time: 7.954553127288818
     9    -6.164888e+01     3.643632e-02
 * time: 8.5161771774292
    10    -6.182973e+01     2.991498e-02
 * time: 9.09042501449585
    11    -6.197747e+01     1.986966e-02
 * time: 9.686761140823364
    12    -6.204180e+01     1.740981e-02
 * time: 10.274598121643066
    13    -6.211182e+01     1.538847e-02
 * time: 10.853779077529907
    14 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671082
    AtomicLocal         -18.8557773
    AtomicNonlocal      14.8522708
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485401 
    Xc                  -19.3336831

    total               -62.261666459162
